# Calibrate Gages

Loop over all gages, calibrate, and export error stats. 

by Mike Durand, 5 March 2025


Dependencies:
* calval reaches dataframe. This notebook assumes that's stored in a calval-data directory
* experiment log dataframe, in this directory
* FLaPE-byrd repo pulled
* various standard Python libraries

This notebook supports the option to either pull SWOT data again (\~10 seconds total runtime per reach) or to analyze data already downloaded (\~ 0.1 seconds runtime/reach). It creates and assumes a directory structure, linked to the experiment log Pandas dataframe.

## 0 Imports & Functions

### 0.1 Imports

In [1]:
import os,sys
from pathlib import Path
import json
import numpy as np
import pandas as pd
from netCDF4 import Dataset
import time

In [2]:
# FLaPE-Byrd imports
sys.path.append('/Users/mtd/GitHub/FLaPE-Byrd') 
from SWOTData import SWOTData,PullSWOT,ReadSWOT
from PullGageData import PullUSGS
from SWOTCalibrationUtilities import   OutputReachCalibration,CalibrateSWOTReach,SetOptions

In [3]:
# this automatically reloads things so you don't have to restart kernel after changing an import
%load_ext autoreload
%autoreload 2

## 1 Set up data and experiment log

### 1.0 Set up directories

In [4]:
CalValDir=Path('calval-data')
GageDir=Path('gage-data')
ExpDataDir=Path('ExperimentData')

### 1.1 Choose experiment and set switches

In [5]:
expdf=pd.read_csv(ExpDataDir.joinpath('experimentlog.csv'),index_col=0)
expdf

expsid    expid                          reachdomain  swotsource slopedata  \
0   exps1   exp1-1  connecticut+willamette calval gages  hydrochron     slope   
1   exps1   exp1-2  connecticut+willamette calval gages  hydrochron    slope2   
2   exps1   exp1-3  connecticut+willamette calval gages  hydrochron     slope   
3   exps1   exp1-4  connecticut+willamette calval gages  hydrochron     slope   
4   exps1   exp1-5  connecticut+willamette calval gages  hydrochron     slope   
5   exps1   exp1-6  connecticut+willamette calval gages  hydrochron     slope   
6   exps1   exp1-7  connecticut+willamette calval gages  hydrochron     slope   
7   exps1   exp1-8  connecticut+willamette calval gages  hydrochron     slope   
8   exps1   exp1-9  connecticut+willamette calval gages  hydrochron     slope   
9   exps1  exp1-10  connecticut+willamette calval gages  hydrochron     slope   
10  exps2   exp2-1                           usgs gages  hydrochron     slope   
11  exps2   exp2-3                           usgs gages  hydrochron     slope   
12  exps2   exp2-4                           usgs gages  hydrochron     slope   
13  exps2   exp2-5                           usgs gages  hydrochron     slope   
14  exps2   exp2-6                           usgs gages  hydrochron     slope   
15  exps2   exp2-8                           usgs gages  hydrochron     slope   
16  exps2   exp2-9                           usgs gages  hydrochron     slope   
17  exps2  exp2-10                           usgs gages  hydrochron     slope   
18  exps2  exp2-11                           usgs gages  hydrochron     slope   
19  exps2  exp2-12                           usgs gages  hydrochron     slope   
20  exps3   exp3-1  connecticut+willamette calval gages     ADT-pvd     slope   

    slopeminimum  slopeconsistencycheck areaopt  constrainhw flowlaw  \
0            NaN               0.000000      fd        False   MWAPN   
1            NaN               0.000000      fd        False   MWAPN   
2       0.000017              -1.000000      fd        False   MWAPN   
3       0.000034              -1.000000      fd        False   MWAPN   
4            NaN               0.000017      fd        False   MWAPN   
5            NaN               0.000000      fh        False   MWAPN   
6            NaN               0.000000      fh         True   MWAPN   
7            NaN               0.000000      fd        False    AHGD   
8            NaN               0.000000      fd        False   MWAPN   
9            NaN               0.000000      fd        False   MWAPN   
10           NaN               0.000000      fd        False   MWAPN   
11      0.000017              -1.000000      fd        False   MWAPN   
12      0.000034              -1.000000      fd        False   MWAPN   
13           NaN               0.000017      fd        False   MWAPN   
14           NaN               0.000000      fh        False   MWAPN   
15           NaN               0.000000      fd        False    AHGD   
16           NaN               0.000000      fd        False   MWAPN   
17           NaN               0.000000      fd        False    AHGW   
18           NaN               0.000000      fd        False    AHGD   
19           NaN               0.000000      fd        False   MWAPN   
20           NaN               0.000000      fd        False   MWAPN   

    darkfracmax  lossfun  
0           0.4   linear  
1           0.4   linear  
2           0.4   linear  
3           0.4   linear  
4           0.4   linear  
5           0.4   linear  
6           0.4   linear  
7           0.4   linear  
8           0.2   linear  
9           0.4  soft_l1  
10          0.4   linear  
11          0.4   linear  
12          0.4   linear  
13          0.4   linear  
14          0.4   linear  
15          0.4   linear  
16          0.2   linear  
17          0.4   linear  
18          0.2   linear  
19          0.4  soft_l1  
20          0.4   linear

In [6]:
#choose experiment index to run
idx=0

In [7]:
# pull local switch settings out of dataframe. this could be improved...
expsid,expid,SWOTSource,fname_pvd,slope_data_element,slope_min,slope_const,area_option,constrainhw,\
    flowlaw,darkfracmax,reachdomain,lossfun=SetOptions(expdf,idx)

running exp1-1
  domain: connecticut+willamette calval gages
  swot data source: hydrochron
  slope data element: slope
  minimum slope applied as filter: nan
  slope consistency check: 0.0
  area option: fd
  constrain height-width option: False
  flow law: MWAPN
  dark frac maximum: 0.4
  loss function: linear


### 1.2 Handle experiment folder

In [8]:
ExpsDir=ExpDataDir.joinpath(expsid)
ExpDir=ExpsDir.joinpath(expid)

In [9]:
# make new experiment set directory if it doesn't exist
if not os.path.isdir(ExpsDir):
    #make new directory
    os.makedirs(ExpsDir)

In [10]:
# make new directory if it doesn't exist
if not os.path.isdir(ExpDir):
    #make new directory
    os.makedirs(ExpDir)

In [11]:
dirname=ExpsDir.joinpath('unfiltered')
if not os.path.isdir(dirname):
    os.makedirs(dirname)    
else:
    print('warning! directory exists. check for old data before continuing')

warning! directory exists. check for old data before continuing


In [12]:
expdirs=['fit+stats','gage+SWOT','gage+SWOT+cal']

for expdir in expdirs:
    dirname=ExpDir.joinpath(expdir)
    if not os.path.isdir(dirname):
        os.makedirs(dirname)    
    else:
        print('warning! directory exists. check for old data before continuing. data will be overwritten')

warning! directory exists. check for old data before continuing. data will be overwritten
warning! directory exists. check for old data before continuing. data will be overwritten
warning! directory exists. check for old data before continuing. data will be overwritten


### 1.3 read in data

In [13]:
if reachdomain == 'connecticut+willamette calval gages':
    # read in list of calval reaches to run on
    fname=CalValDir.joinpath('calvalreaches.json')
elif reachdomain =='usgs gages':
    fname=GageDir.joinpath('gagereaches.json')

with open(fname) as file:
    reaches_to_run=json.load(file)

rids_to_run=list(reaches_to_run.keys())
rids_to_run.sort()
print('there are ', len(rids_to_run),'reaches to run')


there are  13 reaches to run


In [14]:
# read in SWORD
if SWOTSource=='ADT':
    swordfile='/Users/mtd/Data/SWOT/SWORD/SWORD_v17/Reaches_Nodes/netcdf/na_sword_v17.nc'
else:
    swordfile='/Users/mtd/Data/SWOT/SWORD/SWORD_v16/Reaches_Nodes/netcdf/na_sword_v16.nc'

In [15]:
# optionally read in SWORD translation file
if SWOTSource=='ADT':
    swordtranslation=pd.read_csv('/Users/mtd/Data/SWOT/SWORD/SWORD_v17/Reaches_Nodes/v16-17translation/NA_ReachIDs_v17_vs_v16.csv')
    swordtranslation.head()

## 2 Loop over all reaches

SWOTSource:
* Hydrochron, then just pull data down from the web
* ADT, then pull from a dataframe with multiple reaches (need filename)
* offline, then pull from a directory with a dataframe for each reach (need directory)

In [16]:
# get data again by pulling from Hydrochron and USGS? Or just use data already pulled?
GetData=False #if false, then just use the "unfiltered" directory

In [17]:
# open SWORD
sword_dataset = Dataset(swordfile)
swordreachids=sword_dataset["reaches/reach_id"][:]

In [18]:
# set up logging
logdest='file' # screen or file
if logdest=='file':
    logfname=ExpDir.joinpath('logfile.txt')
    logfile=open(logfname,'w')
elif logdest=='screen':
    logfile = sys.stdout # use this option to print logged output to screen

In [19]:
# loop over reaches to run on and calibrate them

start_time = time.perf_counter()

print('starting calibration at',time.asctime(),file=logfile)

nmin=10
n_skip=0
n_run=0
outfreq=10 #prints screen status ever 'outfreq' files
for rid in rids_to_run:    
    # time and counting
    current_time = time.perf_counter()
    execution_time = current_time - start_time
    if (n_run+n_skip) % outfreq == 0:
        print('running ',n_run+n_skip,'/',len(rids_to_run),'execution time so far:',execution_time,'seconds')
    
    
    # just calibrate if a gage is assigned. else skip
    if not reaches_to_run[rid]['agency id']:
        n_skip+=1        
        continue
    usgsid=reaches_to_run[rid]['agency id']
    agency=reaches_to_run[rid]['agency']
    
    
    print('processing SWORD reach',rid,'which corresponds to ',agency,'id',usgsid,file=logfile )
    if SWOTSource=='ADT':
        rid17=int(swordtranslation[swordtranslation['v16_reach_id']==int(rid)]['v17_reach_id'])
        print('  mapping SWORD 16 reach ',rid,'to SWORD17 reach',rid17,file=logfile)
    
    
    fnameu=ExpsDir.joinpath('unfiltered').joinpath(rid+'_SWOT_gage_unfiltered.csv')
    
    if GetData:

        #2 pull USGS data
        QUSGS,nretrv=PullUSGS(usgsid,logfile)
        if nretrv==0:
            print('  no USGS data pulled. skipping',file=logfile)
            n_skip+=1
            continue        

        #3 get SWOT data
        if SWOTSource=='Hydrochron':
            SWOTdict=PullSWOT(rid,logfile)
        elif SWOTSource=='ADT':
            SWOTdict=ReadSWOT(rid,logfile,fname=fname_pvd,reachid17=rid17)

        SWOT=SWOTData(logfile,SWOTdict=SWOTdict)
        SWOT.MatchUSGS(QUSGS)
        # dump dataframe to "unfiltered"        
        SWOT.data['df'].to_csv(fnameu)        
        
    else:
        if not fnameu.exists():
            print('  file not found, skipping', file=logfile )
            n_skip+=1
            continue                    
        # 1-4: read in existing SWOT+gage data from "unfiltered" directory
        SWOT=SWOTData(logfile,SWOTfname=fnameu)
        if SWOTSource=='ADT':
            SWOT.data['reachid17']=rid17

    
    # 4 apply filters  
    if SWOTSource=='ADT':
        idx=list(swordreachids).index(np.int64(SWOT.data['reachid17']))
    else:
        idx=list(swordreachids).index(np.int64(rid))
        
    p_slope=sword_dataset['reaches']['slope'][idx] / 1000 # m/km -> m/m
    print('  SWORD slope=',p_slope*1e5,'cm/km',file=logfile )
    
    SWOT.threshold_filter(slope_min=slope_min,dark_max=darkfracmax)
    SWOT.bitwise_filter()
    SWOT.outlier_test()
    if slope_const >= 0:
        SWOT.slope_limit(p_slope,opt=slope_const)    
    
    # 5 filter data and save to a separate filtered dataframe to be read in by FLaPE-Byrd
    SWOT.data['df']=SWOT.data['df'][~SWOT.data['df']['bad']]
    SWOT.data['df']=SWOT.data['df'][SWOT.data['df']['Qgage'].notna()]
    
    fnameSG=ExpDir.joinpath('gage+SWOT').joinpath(rid+'_SWOT_gage.csv')
    SWOT.data['df'].to_csv(fnameSG)
    
    if len(SWOT.data['df'])<nmin:
        print('  no data remain after filtering. quitting',file=logfile )
        n_skip+=1
        continue    
    
    # 6 calibrate reach and save dataframe
    Cal = CalibrateSWOTReach(fnameSG,logfile,slope_opt=slope_data_element,
                         area_opt=area_option,constrainhw=constrainhw,
                         flowlawname=flowlaw,Verbose=False,lossfun=lossfun)
    SWOT.data['df']['Qhat']=list(Cal.Qhat)    
    # SWOT.data['df'].to_csv('cal_fits/gage+SWOT+cal/'+rid+'_SWOT_gage_cal.csv')    
    fnameSGc=ExpDir.joinpath('gage+SWOT+cal').joinpath(rid+'_SWOT_gage_cal.csv')
    SWOT.data['df'].to_csv(fnameSGc)
    
    # 7 save calibration parameters and error stats
    dfout=OutputReachCalibration(Cal,rid)
    # dfout.to_csv('cal_fits/fit+stats/'+rid+'_fits_stats.csv')    
    fnameout=ExpDir.joinpath('fit+stats').joinpath(rid+'_fits_stats.csv')
    dfout.to_csv(fnameout)
        
    # increment reach counter on success    
    print('  calibration complete.',file=logfile )
    n_run+=1

# close logfile and SWORD
print('Calibrations complete: ',n_run,'calibrations run, and ',n_skip,'skipped')
logfile.close()
sword_dataset.close()

running  0 / 13 execution time so far: 0.000488366000000795 seconds
running  10 / 13 execution time so far: 0.43593982500000017 seconds
Calibrations complete:  9 calibrations run, and  4 skipped
